<a href="https://colab.research.google.com/github/adrian-pieroni/challenge2-data-science-LATAM/blob/main/challenge2_data_science_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción




In [ ]:
import pandas as pd

In [ ]:
# URL de la API que devuelve un JSON
url = '/content/TelecomX_Data.json'

In [ ]:
# Leer directamente desde la URL
df = pd.read_json(url)

###Conocer el conjunto de datos

In [ ]:
# Vista previa de los datos
print(df.head())

   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetService': 'DSL', 'OnlineSecurity': '...   
2  {'InternetService': 'Fiber optic', 'OnlineSecu...   
3  {'I

✅ Ver nombres de columnas:

In [ ]:
df.columns

Index(['customerID', 'Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')

✅ Ver los primeros registros del DataFrame:

In [ ]:
df.head()       # Muestra las primeras 5 filas


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [ ]:
df.tail()       # Muestra las últimas 5 filas

,customerID,Churn,customer,phone,internet,account
7262,9987-LUTYD,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
7263,9992-RRAMN,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7264,9992-UJOEL,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7265,9993-LHIEB,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Two year', 'PaperlessBilling': '..."
7266,9995-HOTOH,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'No', 'MultipleLines': 'No ph...","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Two year', 'PaperlessBilling': '..."


✅ Ver los tipos de datos de cada columna:

In [ ]:
df.dtypes

,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


✅ Resumen completo del DataFrame:

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


✅ Ver un resumen estadístico de las columnas numéricas:

In [ ]:
df.describe(include='object')

,customerID,Churn,customer,phone,internet,account
count,7267,7267,7267,7267,7267,7267
unique,7267,3,891,3,129,6931
top,9995-HOTOH,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'No', 'OnlineSecurity': 'N...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
freq,1,5174,223,3495,1581,6


✅ Identificar las columnas más relevantes para el análisis de evasión.

🔍 ¿Qué contiene cada columna (según los nombres)?

customerID: identificador del cliente (⚠️ no es útil para predicción).

Churn: variable objetivo → 1 si evadió, 0 si no.

customer: probablemente tenga datos demográficos (edad, género, región, etc.).

phone: puede incluir tipo de plan de telefonía, minutos usados, etc.

internet: tipo de servicio de internet, velocidad, uso de datos...

account: info sobre cuenta, pagos, duración del contrato, facturación, etc.

🔍 1. Ver cómo es una fila de cada una:

In [ ]:
print(df['customer'].iloc[0])
print(df['phone'].iloc[0])
print(df['internet'].iloc[0])
print(df['account'].iloc[0])

{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'Yes', 'Dependents': 'Yes', 'tenure': 9}
{'PhoneService': 'Yes', 'MultipleLines': 'No'}
{'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'No', 'TechSupport': 'Yes', 'StreamingTV': 'Yes', 'StreamingMovies': 'No'}
{'Contract': 'One year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}


In [ ]:
# Expandir columnas anidadas y unirlas al DataFrame original
customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

In [ ]:
# Combinar todo en un nuevo DataFrame
df_final = pd.concat([df['customerID'], df['Churn'], customer_df, phone_df, internet_df, account_df], axis=1)


In [ ]:
df_final.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


###Comprobación de incoherencias en los datos

✅ 1. Valores ausentes (NaN o nulos)


In [ ]:
# Conteo de valores nulos por columna
df.isnull().sum()

,0
customerID,0
Churn,0
customer,0
phone,0
internet,0
account,0


In [ ]:
# Porcentaje de valores faltantes por columna
(df.isnull().mean() * 100).round(2)

,0
customerID,0.0
Churn,0.0
customer,0.0
phone,0.0
internet,0.0
account,0.0


✅ 2. Duplicados

In [ ]:
# Ver si hay filas duplicadas
df.duplicated().sum()

TypeError: unhashable type: 'dict'

In [ ]:
# Mostrar las filas duplicadas (si hay)
df[df.duplicated()]

TypeError: unhashable type: 'dict'

✅ 3. Errores de formato (por tipo de dato)

In [ ]:
# Ver tipos de datos
df.dtypes

,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


In [ ]:
# Ver ejemplos de valores "extraños" en columnas numéricas
print(df['TotalCharges'].unique())  # por ejemplo, puede tener strings vacíos o espacios

KeyError: 'TotalCharges'

In [ ]:
# Ver si una columna que debería ser numérica tiene texto
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

KeyError: 'TotalCharges'

✅ 4. Inconsistencias en columnas categóricas

In [ ]:
# Ver valores únicos para todas las columnas categóricas
for col in df.select_dtypes(include='object'):
    print(f"\n{col} - valores únicos:")
    print(df[col].value_counts(dropna=False))


customerID - valores únicos:
customerID
9995-HOTOH    1
0002-ORFBO    1
0003-MKNFE    1
9970-QBCDA    1
9968-FFVVH    1
             ..
0014-BMAQU    1
0013-SMEOE    1
0013-MHZWF    1
0013-EXCHZ    1
0011-IGKFF    1
Name: count, Length: 7267, dtype: int64

Churn - valores únicos:
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

customer - valores únicos:
customer
{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 1}         223
{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 1}       192
{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'Yes', 'Dependents': 'Yes', 'tenure': 72}       77
{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'Yes', 'Dependents': 'Yes', 'tenure': 72}     76
{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 2}        74
                                                                                                ... 
{'gende

✅ 5. Distribución de valores fuera de rango (para numéricas)

In [ ]:
# Estadísticas generales
df.describe()

,customerID,Churn,customer,phone,internet,account
count,7267,7267,7267,7267,7267,7267
unique,7267,3,891,3,129,6931
top,9995-HOTOH,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'No', 'OnlineSecurity': 'N...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
freq,1,5174,223,3495,1581,6


✅ ✅ VERSIÓN CORREGIDA DEFINITIVA:

In [ ]:
# Filtrar columnas que no contienen diccionarios
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]

# Crear resumen de calidad de datos
resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Calcular duplicados usando solo columnas válidas
duplicados = df[columnas_validas].duplicated().sum()

# Añadir fila con el número de filas duplicadas
resumen.loc['Duplicados (filas)', ['Tipo', 'Nulos (%)', 'Únicos']] = [None, None, duplicados]

# Mostrar el resumen
resumen

,Tipo,Nulos (%),Únicos
customerID,object,0.0,7267.0
Churn,object,0.0,3.0
Duplicados (filas),None,NaN,0.0


✅ 6. Chequeo de columnas constantes o casi vacías

In [ ]:
# Detectar columnas sin diccionarios
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]

# Aplicar nunique solo a esas columnas
df[columnas_validas].nunique().sort_values()

,0
Churn,3
customerID,7267


📌 Resumen rápido de calidad de datos

In [ ]:
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]

In [ ]:
# Crear el resumen solo con columnas hashables
resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Calcular duplicados usando solo columnas válidas
duplicados = df[columnas_validas].duplicated().sum()

# Agregar fila al final con duplicados
resumen.loc['Duplicados (filas)', ['Tipo', 'Nulos (%)', 'Únicos']] = [None, None, duplicados]

resumen

,Tipo,Nulos (%),Únicos
customerID,object,0.0,7267.0
Churn,object,0.0,3.0
Duplicados (filas),None,NaN,0.0


In [ ]:
df.isnull().sum()              # Conteo

,0
customerID,0
Churn,0
customer,0
phone,0
internet,0
account,0


In [ ]:
(df.isnull().mean() * 100)     # Porcentaje

,0
customerID,0.0
Churn,0.0
customer,0.0
phone,0.0
internet,0.0
account,0.0


In [ ]:
df.isnull().sum()

,0
customerID,0
Churn,0
customer,0
phone,0
internet,0
account,0


#🔧 Transformación



In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

KeyError: 'TotalCharges'

In [ ]:
print(df.columns.tolist())

['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']


In [ ]:
import requests
import pandas as pd

# URL de la API (reemplaza con la verdadera URL de Telecom X)
url = 'https://api.telecomx.com/clientes'  # ejemplo ficticio

# Si necesitas encabezados, como un token de autorización, agrégalo aquí
headers = {
    'Authorization': 'Bearer TU_TOKEN_AQUI',  # si es necesario
    'Accept': 'application/json'
}

# Solicitar los datos desde la API
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data_json = response.json()
    # Convertir a DataFrame (ajusta según la estructura del JSON)
    df = pd.DataFrame(data_json)
    print("Datos importados correctamente. Vista previa:")
    print(df.head())
else:
    print(f"Error al acceder a la API. Código de estado: {response.status_code}")
Notas:
Si el JSON tiene una estructura anidada (por ejemplo, {"clientes": [{...}, {...}]}), podrías necesitar hacer algo como pd.DataFrame(data_json["clientes"]).

Si el acceso a la API es paginado (es decir, entrega los datos en varias partes), también podrías necesitar hacer múltiples solicitudes en un bucle.

Si me compartís la URL (o un ejemplo de la estructura del JSON), puedo darte un código más preciso.








Dijiste:
como sería con el comando read?


ChatGPT dijo:
Perfecto. Si la API de Telecom X permite acceso directo a través de una URL pública o privada que entrega un JSON sin necesidad de autenticación compleja (es decir, puedes acceder directamente con un GET), entonces podés usar pandas.read_json() directamente, así:

python
Copiar
Editar
import pandas as pd

# URL de la API que devuelve un JSON
url = 'https://api.telecomx.com/clientes'  # ejemplo ficticio

# Leer directamente desde la URL
df = pd.read_json(url)

# Vista previa de los datos
print("Datos importados correctamente:")
print(df.head())
Tené en cuenta:
read_json() funciona solo si el JSON está bien estructurado y es accesible directamente desde una URL (como si fuese un archivo JSON plano).

Si la respuesta es anidada (por ejemplo, tiene una clave raíz tipo {"data": [...]}), tendrías que hacer algo como:

python
Copiar
Editar
import requests
import pandas as pd

url = 'https://api.telecomx.com/clientes'

response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()
    df = pd.DataFrame(json_data['data'])  # reemplaza 'data' por la clave correcta
    print(df.head())
else:
    print(f"Error: {response.status_code}")
¿Querés compartir un ejemplo del JSON o el formato de la URL para darte una versión definitiva?








Dijiste:
Con qué comandos puedo explorar las columnas del dataset y verificar sus tipos de datos?


ChatGPT dijo:
Excelente pregunta. Una vez que ya cargaste el DataFrame en tu entorno de Google Colab, podés explorar las columnas y sus tipos de datos con estos comandos de pandas:

✅ Ver nombres de columnas:
python
Copiar
Editar
df.columns
✅ Ver los primeros registros del DataFrame:
python
Copiar
Editar
df.head()       # Muestra las primeras 5 filas
df.head(10)     # Muestra las primeras 10 filas (puedes cambiar el número)
✅ Ver los tipos de datos de cada columna:
python
Copiar
Editar
df.dtypes
✅ Resumen completo del DataFrame:
python
Copiar
Editar
df.info()
Esto incluye:

número de filas y columnas,

nombres de columnas,

tipo de datos,

cantidad de valores no nulos por columna.

✅ Ver un resumen estadístico de las columnas numéricas:
python
Copiar
Editar
df.describe()
Para columnas categóricas (como strings), podés usar:

python
Copiar
Editar
df.describe(include='object')
¿Querés que te arme un pequeño bloque de código con todos estos pasos para ejecutar juntos?







Dijiste:
con qué comandos puedo  Consultar el diccionario para comprender mejor el significado de las variabl


ChatGPT dijo:
Muy buena pregunta, Adrian. En ciencia de datos, el "diccionario de datos" (o data dictionary) no está incluido automáticamente en un DataFrame de pandas, a menos que el proveedor de datos lo haya incluido como un archivo adicional o en los metadatos.

🧩 Posibles formas de consultar el diccionario de datos:
✅ 1. Si el diccionario viene como otro archivo (por ejemplo, CSV, Excel, o JSON)
Consultalo como cualquier otro dataset:

python
Copiar
Editar
diccionario = pd.read_csv('ruta_al_diccionario.csv')
print(diccionario.head())
En general, suele tener columnas como:

nombre_variable

descripcion

tipo_dato

valores_posibles o observaciones

✅ 2. Si el diccionario está embebido en el JSON o la API
A veces, el JSON que devuelve la API tiene una estructura como esta:

json
Copiar
Editar
{
  "metadata": {
    "variables": [
      {"nombre": "edad", "descripcion": "Edad del cliente en años"},
      {"nombre": "estado_evasion", "descripcion": "1 si el cliente evadió el pago, 0 si no"},
      ...
    ]
  },
  "data": [...]
}
En ese caso, accedés así:

python
Copiar
Editar
response = requests.get(url)
json_data = response.json()

# Acceder al diccionario
diccionario = pd.DataFrame(json_data['metadata']['variables'])
print(diccionario)
✅ 3. Si no tenés el diccionario pero querés construir uno básico a mano
Podés crearlo tú mismo si entendés las variables:

python
Copiar
Editar
diccionario = {
    'edad': 'Edad del cliente en años',
    'sexo': 'Sexo biológico del cliente',
    'servicio': 'Tipo de servicio contratado',
    'estado_evasion': 'Indica si el cliente evadió el pago (1 = sí, 0 = no)',
    # y así con cada columna...
}

# Consultar descripción de una variable:
print(diccionario['servicio'])
✅ 4. Si no hay diccionario disponible y necesitás inferir a partir de los datos
Podés inspeccionar valores únicos de las variables categóricas:

python
Copiar
Editar
for col in df.select_dtypes(include='object'):
    print(f"{col}: {df[col].unique()}")
¿Tenés acceso a un archivo separado con el diccionario, o querés ayuda para inferirlo a partir de los datos actuales?








Dijiste:
como importo el marchivo json para que quede disponible siempre en googleColab?


ChatGPT dijo:
Muy buena pregunta. En Google Colab, los archivos que subís manualmente se almacenan temporalmente: se eliminan cuando se reinicia el entorno. Pero tenés varias opciones para que tu archivo JSON permanezca disponible o sea fácil de volver a cargar.

✅ OPCIÓN 1: Guardar el JSON en tu Google Drive (RECOMENDADA)
Esto te permite tener acceso permanente al archivo cada vez que abras Colab.

Paso 1: Montar Google Drive en Colab
python
Copiar
Editar
from google.colab import drive
drive.mount('/content/drive')
Paso 2: Leer el archivo JSON desde Drive
Supongamos que lo tenés en:
/content/drive/MyDrive/datos/telecom_clientes.json

python
Copiar
Editar
import pandas as pd

ruta = '/content/drive/MyDrive/datos/telecom_clientes.json'
df = pd.read_json(ruta)

print(df.head())
✅ OPCIÓN 2: Subir manualmente el archivo JSON (solo mientras dure la sesión)
Si solo lo necesitás una vez por sesión:

Paso 1: Subir el archivo
python
Copiar
Editar
from google.colab import files
uploaded = files.upload()
Te pedirá seleccionar el archivo desde tu PC.

Paso 2: Leerlo con pandas
python
Copiar
Editar
import pandas as pd

df = pd.read_json('telecom_clientes.json')  # nombre exacto del archivo
print(df.head())
✅ OPCIÓN 3: Leerlo desde una URL (siempre disponible online)
Si tenés el JSON en una URL pública o de una API, podés hacer esto:

python
Copiar
Editar
url = 'https://tu-servidor.com/archivo.json'
df = pd.read_json(url)
¿Cuál elegís?
Si querés acceso permanente y fácil: usa Google Drive.

Si el archivo es temporal: subida manual.

Si el JSON está online: cargar por URL.

¿Querés que te ayude a configurar alguna de estas opciones con tu archivo real?








Dijiste:
con que comandos puedo Identificar las columnas más relevantes para el análisis de evasión.


ChatGPT dijo:
Para identificar las columnas más relevantes para analizar la evasión, podés usar una combinación de exploración visual, análisis estadístico y modelado. Acá te muestro un enfoque práctico en Google Colab con Python y pandas, suponiendo que tu variable objetivo es algo como estado_evasion (1 si el cliente evadió, 0 si no).

🧭 PASO A PASO PARA IDENTIFICAR VARIABLES RELEVANTES
✅ 1. Verificar correlaciones con la variable de evasión (para columnas numéricas)
python
Copiar
Editar
import seaborn as sns
import matplotlib.pyplot as plt

# Correlación solo para variables numéricas
correlaciones = df.corr(numeric_only=True)
plt.figure(figsize=(10,6))
sns.heatmap(correlaciones[['estado_evasion']].sort_values(by='estado_evasion', ascending=False), annot=True, cmap='coolwarm')
plt.title('Correlación con estado_evasion')
plt.show()
✅ 2. Analizar variables categóricas por proporción de evasión
Esto te muestra si, por ejemplo, ciertos tipos de servicio o regiones tienen mayor evasión:

python
Copiar
Editar
# Reemplaza 'tipo_servicio', 'region', etc. por tus columnas categóricas reales
columnas_categoricas = ['tipo_servicio', 'region', 'sexo', 'plan']

for col in columnas_categoricas:
    print(f"\nProporción de evasión por categoría en '{col}':")
    print(df.groupby(col)['estado_evasion'].mean().sort_values(ascending=False))
✅ 3. Usar un modelo simple para detectar importancia de variables (ej. Random Forest)
Esto te da una medida automática de qué columnas son más predictivas de la evasión:

python
Copiar
Editar
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Preparamos los datos (con variables numéricas y codificadas)
df_model = df.copy()
df_model = pd.get_dummies(df_model, drop_first=True)  # convierte categóricas en dummies

X = df_model.drop('estado_evasion', axis=1)
y = df_model['estado_evasion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Importancia de características
importancias = pd.Series(modelo.feature_importances_, index=X.columns)
importantes = importancias.sort_values(ascending=False).head(15)

# Gráfico
importantes.plot(kind='barh', figsize=(10,6))
plt.title('Importancia de variables para predecir evasión')
plt.gca().invert_yaxis()
plt.show()
💡 Sugerencia general:
Variables demográficas (edad, región, sexo) y de servicio (plan, tipo de conexión, antigüedad) suelen ser relevantes para evasión.

Si tenés variables como "uso mensual", "factura promedio", "interacciones con soporte", también podrían ser predictoras.

¿Querés que trabajemos con tu dataset directamente? Podés decirme cómo se llama la columna de evasión y algunas variables clave.








Dijiste:
Index(['customerID', 'Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')


ChatGPT dijo:
¡Gracias! Veo que tu DataFrame tiene estas columnas:

pgsql
Copiar
Editar
Index(['customerID', 'Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')
Parece que estás trabajando con un dataset de tipo churn/evasión (como el clásico de telecomunicaciones). Vamos a analizar qué columnas son relevantes para predecir Churn (que es tu variable objetivo de evasión).

🔍 ¿Qué contiene cada columna (según los nombres)?
customerID: identificador del cliente (⚠️ no es útil para predicción).

Churn: variable objetivo → 1 si evadió, 0 si no.

customer: probablemente tenga datos demográficos (edad, género, región, etc.).

phone: puede incluir tipo de plan de telefonía, minutos usados, etc.

internet: tipo de servicio de internet, velocidad, uso de datos...

account: info sobre cuenta, pagos, duración del contrato, facturación, etc.

✅ Siguiente paso: Expandir esas columnas
Al parecer, columnas como customer, phone, etc. podrían estar en formato anidado (es decir, cada celda contiene un diccionario o subestructura). Verifiquémoslo:

🔍 1. Ver cómo es una fila de cada una:
python
Copiar
Editar
print(df['customer'].iloc[0])
print(df['phone'].iloc[0])
print(df['internet'].iloc[0])
print(df['account'].iloc[0])
Si te devuelve algo como:

python
Copiar
Editar
{'gender': 'Female', 'age': 45, 'region': 'South'}
Entonces podemos expandirlas:

🔧 2. Expandir columnas anidadas (si es necesario)
python
Copiar
Editar
# Expandir columnas anidadas y unirlas al DataFrame original
customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

# Combinar todo en un nuevo DataFrame
df_final = pd.concat([df['customerID'], df['Churn'], customer_df, phone_df, internet_df, account_df], axis=1)
🎯 3. Identificar variables relevantes para Churn
Ahora que tenés todas las variables expandidas en df_final, podés seguir con el análisis sugerido antes:

Correlaciones (si hay variables numéricas)

Proporción de churn por categoría

Importancia de variables con Random Forest

¿Querés que lo apliquemos directamente paso a paso con este df_final? También me podés pegar una fila de df['customer'].iloc[0] para ayudarte a expandirla correctamente.








Dijiste:
Con qué comandos puedo Identificar las columnas más relevantes para el análisis de evasión.


ChatGPT dijo:
Perfecto, Adrian. Para identificar las columnas más relevantes para analizar la evasión (Churn) en tu dataset, podés seguir estos 3 enfoques complementarios desde pandas + scikit-learn en Google Colab.

✅ 1. Análisis Exploratorio: Variables categóricas y su relación con Churn
Este enfoque te permite ver si ciertas categorías tienen mayor proporción de evasión.

python
Copiar
Editar
columnas_categoricas = df.select_dtypes(include='object').columns.drop(['customerID', 'Churn'])

for col in columnas_categoricas:
    print(f"\n{col} - Tasa de evasión por categoría:")
    print(df.groupby(col)['Churn'].value_counts(normalize=True).unstack()[1].sort_values(ascending=False))
💡 Interpreta: si una categoría tiene tasa alta de Churn = 1, es potencialmente relevante.

✅ 2. Correlación con Churn (para variables numéricas)
Ideal para detectar relaciones lineales con Churn.

python
Copiar
Editar
# Asegurate de que 'Churn' sea numérica (1 o 0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})  # si no lo hiciste antes

# Ver correlaciones con variables numéricas
correlaciones = df.corr(numeric_only=True)
correlaciones_churn = correlaciones['Churn'].drop('Churn').sort_values(ascending=False)
print(correlaciones_churn)
✅ 3. Modelo predictivo: Importancia de variables (Random Forest)
Este método te da una medida cuantitativa de importancia basada en la capacidad predictiva.

python
Copiar
Editar
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd

# Copia del DataFrame y codificación de variables categóricas
df_model = df.copy()
df_model = df_model.drop('customerID', axis=1)

# Codificar variables categóricas
for col in df_model.select_dtypes(include='object').columns:
    df_model[col] = LabelEncoder().fit_transform(df_model[col])

# Separar features y target
X = df_model.drop('Churn', axis=1)
y = df_model['Churn']

# Entrenar modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Importancia de variables
importancias = pd.Series(modelo.feature_importances_, index=X.columns)
importancias.sort_values(ascending=True).plot(kind='barh', figsize=(10,8))
plt.title("Importancia de variables para evasión (Churn)")
plt.show()
✅ BONUS: Selección automática con SelectKBest
python
Copiar
Editar
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)
selected_columns = X.columns[selector.get_support()]
print("Top variables seleccionadas:", selected_columns.tolist())
¿Querés que te ayude a ejecutar alguno de estos pasos directamente con tu df actual? Solo confirmame si Churn está en formato Yes/No o 1/0.








Dijiste:
con que comandos podría verificar si hay problemas en los datos que puedan afectar el anális: valores ausentes, duplicados, errores de formato e inconsistencias en las categorías, para asegurarme de que los datos estén listos para las siguientes etapas.


ChatGPT dijo:
Excelente paso, Adrian. Verificar la calidad de los datos antes de analizarlos o modelarlos es clave. Te dejo un conjunto de comandos en pandas que te van a permitir detectar problemas comunes:

✅ 1. Valores ausentes (NaN o nulos)
python
Copiar
Editar
# Conteo de valores nulos por columna
df.isnull().sum()

# Porcentaje de valores faltantes por columna
(df.isnull().mean() * 100).round(2)
💡 Si hay columnas con muchos nulos (>30%), evaluá si deben eliminarse o imputarse.

✅ 2. Duplicados
python
Copiar
Editar
# Ver si hay filas duplicadas
df.duplicated().sum()

# Mostrar las filas duplicadas (si hay)
df[df.duplicated()]
✅ 3. Errores de formato (por tipo de dato)
python
Copiar
Editar
# Ver tipos de datos
df.dtypes

# Ver ejemplos de valores "extraños" en columnas numéricas
print(df['TotalCharges'].unique())  # por ejemplo, puede tener strings vacíos o espacios

# Ver si una columna que debería ser numérica tiene texto
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
✅ 4. Inconsistencias en columnas categóricas
python
Copiar
Editar
# Ver valores únicos para todas las columnas categóricas
for col in df.select_dtypes(include='object'):
    print(f"\n{col} - valores únicos:")
    print(df[col].value_counts(dropna=False))
💡 Esto te ayuda a detectar valores mal escritos como 'No', 'no', ' NO ', o espacios vacíos.

✅ 5. Distribución de valores fuera de rango (para numéricas)
python
Copiar
Editar
# Estadísticas generales
df.describe()

# Ver posibles outliers en una columna
import seaborn as sns
sns.boxplot(x=df['MonthlyCharges'])  # cambiar por la variable deseada
✅ 6. Chequeo de columnas constantes o casi vacías
python
Copiar
Editar
# Columnas con un solo valor único (no aportan información)
df.nunique().sort_values()
📌 TIP EXTRA: Resumen rápido de calidad de datos
Podés hacer un resumen rápido así:

python
Copiar
Editar
resumen = pd.DataFrame({
    'Tipo': df.dtypes,
    'Nulos (%)': df.isnull().mean() * 100,
    'Únicos': df.nunique(),
    'Duplicados': df.duplicated().sum()
})

print(resumen)
¿Querés que armemos una función que automatice este diagnóstico en tu dataset? Puedo darte un snippet listo para correr.








Dijiste:
TypeError                                 Traceback (most recent call last)
<ipython-input-56-1780948758> in <cell line: 0>()
      2     'Tipo': df.dtypes,
      3     'Nulos (%)': df.isnull().mean() * 100,
----> 4     'Únicos': df.nunique(),
      5     'Duplicados': df.duplicated().sum()
      6 })

9 frames
/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py in unique_with_mask(values, mask)
    438     table = hashtable(len(values))
    439     if mask is None:
--> 440         uniques = table.unique(values)
    441         uniques = _reconstruct_data(uniques, original.dtype, original)
    442         return uniques

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.unique()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable._unique()

TypeError: unhashable type: 'dict'


ChatGPT dijo:
Gracias por compartir el error, Adrian. Ese TypeError: unhashable type: 'dict' indica que alguna columna de tu DataFrame contiene valores que son diccionarios (dict), lo cual no es compatible con algunas operaciones como nunique() o duplicated().

✅ ¿Qué está pasando?
Cuando tenés columnas anidadas (como las que mencionaste antes: 'customer', 'phone', 'internet', 'account'), cada celda puede contener un diccionario JSON, lo que causa errores en funciones que necesitan comparar o contar valores (porque los dict no son hashables en Python).

🔧 SOLUCIÓN: Excluir columnas con diccionarios antes de aplicar nunique() y duplicated()
🔁 Versión corregida del resumen:
python
Copiar
Editar
# Filtrar columnas que NO tienen diccionarios
columnas_validas = [col for col in df.columns if not isinstance(df[col].iloc[0], dict)]

# Crear resumen solo con columnas válidas
resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Agregar total de duplicados al final
resumen.loc['Duplicados (filas)', 'Únicos'] = df.duplicated().sum()

resumen
✅ ¿Y qué hago con las columnas con diccionarios?
Podés expanderlas con json_normalize, como mencionamos antes:

python
Copiar
Editar
from pandas import json_normalize

# Ejemplo con la columna 'customer'
customer_df = json_normalize(df['customer'])
phone_df = json_normalize(df['phone'])
# ... y así con las demás

# Luego, unís todo en un único DataFrame limpio
df_limpio = pd.concat([df.drop(['customer', 'phone', 'internet', 'account'], axis=1),
                       customer_df, phone_df], axis=1)
¿Querés que te prepare un script completo para limpiar y preparar ese dataset automáticamente? Puedo ayudarte a dejarlo listo para análisis y modelos.








Dijiste:
TypeError                                 Traceback (most recent call last)
<ipython-input-56-1780948758> in <cell line: 0>()
      2     'Tipo': df.dtypes,
      3     'Nulos (%)': df.isnull().mean() * 100,
----> 4     'Únicos': df.nunique(),
      5     'Duplicados': df.duplicated().sum()
      6 })

9 frames
/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py in unique_with_mask(values, mask)
    438     table = hashtable(len(values))
    439     if mask is None:
--> 440         uniques = table.unique(values)
    441         uniques = _reconstruct_data(uniques, original.dtype, original)
    442         return uniques

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.unique()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable._unique()


ChatGPT dijo:
Gracias, Adrian. Veo que el mismo error persiste porque la línea df.nunique() sigue intentando ejecutarse sobre todas las columnas, incluyendo aquellas que contienen diccionarios (dict), lo cual no está permitido.

Vamos a resolverlo paso a paso y de forma segura, evitando cualquier columna con valores no analizables.

✅ ✅ VERSIÓN CORREGIDA DEFINITIVA:
python
Copiar
Editar
# Filtrar solo las columnas que no contienen diccionarios u objetos complejos
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]

# Crear un resumen solo con columnas válidas
resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Agregar total de duplicados de todo el DataFrame al final del resumen
resumen.loc['Duplicados (filas)', 'Únicos'] = df.duplicated().sum()

resumen
🔍 ¿Qué hicimos diferente?
Usamos:

python
Copiar
Editar
df[col].dropna().iloc[0]
para evitar errores si la primera celda es NaN.

Nos quedamos solo con las columnas donde el primer valor no es un dict.

Esto te va a dar un resumen confiable de calidad de datos, y evitará que funciones como nunique() fallen.

¿Querés que te prepare un script para también expandir automáticamente las columnas con diccionarios (customer, phone, etc.) y combinarlas en un solo DataFrame limpio?








Dijiste:
TypeError                                 Traceback (most recent call last)
<ipython-input-57-3172971586> in <cell line: 0>()
     10
     11 # Agregar total de duplicados de todo el DataFrame al final del resumen
---> 12 resumen.loc['Duplicados (filas)', 'Únicos'] = df.duplicated().sum()

3 frames
/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py in factorize_array(values, use_na_sentinel, size_hint, na_value, mask)
    593
    594     table = hash_klass(size_hint or len(values))
--> 595     uniques, codes = table.factorize(
    596         values,
    597         na_sentinel=-1,

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.factorize()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable._unique()

TypeError: unhashable type: 'dict'


ChatGPT dijo:
Gracias por tu paciencia, Adrian. El error persiste porque df.duplicated() también falla cuando el DataFrame contiene columnas con diccionarios (dict), ya que no se pueden comparar directamente ni usar para identificar duplicados.

✅ Solución definitiva
Vamos a calcular las filas duplicadas usando solo columnas válidas (que no contienen diccionarios), igual que hicimos con nunique().

🔧 Código corregido y funcional:
python
Copiar
Editar
# Filtrar columnas que no contienen diccionarios
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]

# Crear resumen de calidad de datos
resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Calcular duplicados usando solo columnas válidas
duplicados = df[columnas_validas].duplicated().sum()

# Añadir fila con el número de filas duplicadas
resumen.loc['Duplicados (filas)', ['Tipo', 'Nulos (%)', 'Únicos']] = [None, None, duplicados]

# Mostrar el resumen
resumen
📌 ¿Qué resuelve?
Ya no intenta comparar ni contar columnas con dict.

La fila Duplicados (filas) te muestra cuántas filas están duplicadas considerando solo las columnas comparables.

No genera más TypeError.

Si querés, puedo ayudarte a limpiar o expandir las columnas con dict (customer, phone, etc.) para tener un dataset plano y listo para modelar. ¿Querés que avancemos con eso?








Dijiste:
TypeError                                 Traceback (most recent call last)
<ipython-input-60-552644055> in <cell line: 0>()
      1 # Columnas con un solo valor único (no aportan información)
----> 2 df.nunique().sort_values()

9 frames
/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py in unique_with_mask(values, mask)
    438     table = hashtable(len(values))
    439     if mask is None:
--> 440         uniques = table.unique(values)
    441         uniques = _reconstruct_data(uniques, original.dtype, original)
    442         return uniques

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.unique()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable._unique()

TypeError: unhashable type: 'dict'


ChatGPT dijo:
El error sigue ocurriendo por el mismo motivo: df.nunique() intenta analizar columnas que contienen valores de tipo dict, los cuales no son hashables y no se pueden contar directamente en pandas.

✅ SOLUCIÓN FINAL: Excluir columnas problemáticas antes de usar .nunique()
Vamos a identificar automáticamente las columnas que no contienen diccionarios y aplicar nunique() solo a ellas.

🔧 Código corregido:
python
Copiar
Editar
# Detectar columnas sin diccionarios
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]

# Aplicar nunique solo a esas columnas
df[columnas_validas].nunique().sort_values()
✅ EXTRA: ¿Querés saber cuáles columnas tienen dict?
Si querés ver exactamente cuáles están causando problemas:

python
Copiar
Editar
columnas_con_dict = [col for col in df.columns if isinstance(df[col].dropna().iloc[0], dict)]
print("Columnas con diccionarios:", columnas_con_dict)
Una vez que identifiques esas columnas (probablemente 'customer', 'phone', 'internet', 'account'), podés expanderlas con json_normalize.

¿Querés que te muestre cómo expandirlas todas y unirlas al DataFrame original, dejándolo limpio y listo para análisis?








Dijiste:
TypeError                                 Traceback (most recent call last)
<ipython-input-63-4160345697> in <cell line: 0>()
      2     'Tipo': df.dtypes,
      3     'Nulos (%)': df.isnull().mean() * 100,
----> 4     'Unicos': df.nunique(),
      5     'Duplicados': df.duplicated().sum()
      6 })

9 frames
/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py in unique_with_mask(values, mask)
    438     table = hashtable(len(values))
    439     if mask is None:
--> 440         uniques = table.unique(values)
    441         uniques = _reconstruct_data(uniques, original.dtype, original)
    442         return uniques

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.unique()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable._unique()

TypeError: unhashable type: 'dict'


ChatGPT dijo:
Gracias por insistir, Adrian. El error sigue porque estás aplicando .nunique() directamente a df, que aún contiene columnas con diccionarios, lo cual rompe la función porque intenta contar valores no comparables.

Vamos a resolverlo paso a paso y limpio, garantizando que nunca se analicen columnas problemáticas.

✅ PASO 1: Identificar columnas sin diccionarios (hashables)
python
Copiar
Editar
columnas_validas = [col for col in df.columns if not isinstance(df[col].dropna().iloc[0], dict)]
✅ PASO 2: Crear resumen confiable
python
Copiar
Editar
# Crear el resumen solo con columnas hashables
resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Calcular duplicados usando solo columnas válidas
duplicados = df[columnas_validas].duplicated().sum()

# Agregar fila al final con duplicados
resumen.loc['Duplicados (filas)', ['Tipo', 'Nulos (%)', 'Únicos']] = [None, None, duplicados]

resumen
✅ ¿Querés ir un paso más?
Te puedo dar un código completo para:

Expandir las columnas con diccionarios (customer, phone, etc.).

Unir todo en un DataFrame limpio.

Volver a correr todo este resumen sin errores.

¿Te gustaría?








Dijiste:
como puedo hacer para identificar las inconsistencias?


ChatGPT dijo:
Buena pregunta, Adrian. Identificar inconsistencias en los datos es una parte clave del preprocesamiento para asegurar que tus análisis (por ejemplo, sobre evasión) sean confiables. Te muestro cómo detectarlas por tipo:

✅ 1. Valores faltantes
Ya lo viste antes, pero lo repetimos por claridad:

python
Copiar
Editar
df.isnull().sum()              # Conteo
(df.isnull().mean() * 100)     # Porcentaje
✅ 2. Duplicados
python
Copiar
Editar
df.duplicated().sum()          # Número de filas duplicadas
df[df.duplicated()]            # Ver duplicados
✅ 3. Errores de tipo de dato
Por ejemplo, si una columna numérica contiene texto:

python
Copiar
Editar
df.dtypes                      # Verifica los tipos
df['TotalCharges'].unique()   # Valores inesperados
Para convertir y corregir:

python
Copiar
Editar
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
✅ 4. Valores categóricos inconsistentes
Ideal para columnas como gender, internetService, paymentMethod, etc.:

python
Copiar
Editar
# Ver valores únicos de todas las columnas categóricas
for col in df.select_dtypes(include='object'):
    print(f"\n{col}:", df[col].unique())
⚠️ Te permite detectar casos como:

'Yes', 'yes', ' YES ' → mal escritos o con espacios.

'No internet' y 'No' usados como si fueran lo mismo.

Valores vacíos ('') en lugar de NaN.

Puedes limpiar con:

python
Copiar
Editar
df['internetService'] = df['internetService'].str.strip().str.lower()
✅ 5. Columnas con un solo valor
No aportan información, pueden eliminarse:

python
Copiar
Editar
df.nunique().sort_values()
✅ 6. Outliers en columnas numéricas
python
Copiar
Editar
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df['MonthlyCharges'])
plt.show()
✅ 7. Errores lógicos
Ejemplos:

Cliente sin servicio de internet pero con cargos por streaming.

Cliente con tenure = 0 y MonthlyCharges > 0.

Ejemplo de chequeo lógico:

python
Copiar
Editar
df[(df['internetService'] == 'No') & (df['streamingTV'] == 'Yes')]
📌 ¿Querés un chequeo automático?
Puedo darte un script que:

Revisa nulos

Encuentra columnas constantes

Detecta errores lógicos frecuentes en este tipo de datasets (evasión / churn)

¿Querés que lo genere?








Dijiste:
Como puedo realizar las correcciones necesarias?


ChatGPT dijo:
Excelente, Adrian. Una vez identificadas las inconsistencias, el siguiente paso es corregir los datos para que estén listos para análisis o modelado. A continuación, te muestro cómo abordar cada tipo de problema con soluciones concretas en código.

✅ 1. Valores faltantes (missing values)
➤ Ver cuántos hay:
python
Copiar
Editar
df.isnull().sum()
➤ Opciones para corregir:
🔹 Eliminar filas (si son pocas):
python
Copiar
Editar
df = df.dropna()
🔹 Rellenar con un valor:
python
Copiar
Editar
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df['internetService'] = df['internetService'].fillna('Desconocido')
🔹 Rellenar con la media, mediana o moda:
python
Copiar
Editar
df['MonthlyCharges'] = df['MonthlyCharges'].fillna(df['MonthlyCharges'].mean())
✅ 2. Duplicados
➤ Eliminar filas duplicadas:
python
Copiar
Editar
df = df.drop_duplicates()
✅ 3. Errores de formato o tipo de datos
➤ Convertir texto a numérico:
python
Copiar
Editar
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
➤ Uniformar texto (por ejemplo: "yes", "YES", " Yes "):
python
Copiar
Editar
df['Churn'] = df['Churn'].str.strip().str.lower()
✅ 4. Valores categóricos inconsistentes
➤ Inspeccionar valores únicos:
python
Copiar
Editar
for col in df.select_dtypes(include='object'):
    print(f"{col}: {df[col].unique()}")
➤ Corregir o reagrupar valores:
python
Copiar
Editar
# Unificar 'no internet service' y 'no' como 'no'
df.replace({'no internet service': 'no', 'No internet': 'no'}, inplace=True)

# Corregir valores escritos de forma inconsistente
df['paymentMethod'] = df['paymentMethod'].replace({
    'Bank transfer (automatic)': 'bank_transfer',
    'Mailed check': 'mail_check'
})
✅ 5. Columnas sin variabilidad (constantes)
➤ Eliminar columnas con un solo valor:
python
Copiar
Editar
for col in df.columns:
    if df[col].nunique() == 1:
        df.drop(col, axis=1, inplace=True)
✅ 6. Errores lógicos
➤ Ejemplo: cliente sin internet pero con servicio de streaming
python
Copiar
Editar
# Reemplazar inconsistencias
cond = (df['internetService'] == 'No') & (df['streamingTV'] == 'Yes')
df.loc[cond, 'streamingTV'] = 'No'
✅ 7. Outliers
➤ Revisión rápida visual:
python
Copiar
Editar
import seaborn as sns
sns.boxplot(x=df['MonthlyCharges'])
➤ Eliminar outliers extremos (ejemplo con IQR):
python
Copiar
Editar
q1 = df['MonthlyCharges'].quantile(0.25)
q3 = df['MonthlyCharges'].quantile(0.75)
iqr = q3 - q1

filtro = (df['MonthlyCharges'] >= (q1 - 1.5 * iqr)) & (df['MonthlyCharges'] <= (q3 + 1.5 * iqr))
df = df[filtro]
🎁 ¿Querés un script automático?
Puedo generarte un código completo que:

Limpie valores nulos y duplicados

Normalice texto

Expanda columnas JSON si aún no lo hiciste

Corrija errores lógicos comunes

¿Querés que lo prepare?








Dijiste:
KeyError                                  Traceback (most recent call last)
/usr/local/lib/python3.11/dist-packages/pandas/core/indexes/base.py in get_loc(self, key)
   3804         try:
-> 3805             return self._engine.get_loc(casted_key)
   3806         except KeyError as err:

index.pyx in pandas._libs.index.IndexEngine.get_loc()

index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'TotalCharges'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
2 frames
/usr/local/lib/python3.11/dist-packages/pandas/core/indexes/base.py in get_loc(self, key)
   3810             ):
   3811                 raise InvalidIndexError(key)
-> 3812             raise KeyError(key) from err
   3813         except TypeError:
   3814             # If we have a listlike key, _check_indexing_error will raise

KeyError: 'TotalCharges'


ChatGPT dijo:
Este error KeyError: 'TotalCharges' indica que no existe una columna llamada 'TotalCharges' en tu DataFrame df.

✅ Posibles causas y soluciones
🔍 1. ¿El nombre de la columna es diferente?
Verificá los nombres reales de las columnas:

python
Copiar
Editar
print(df.columns.tolist())
Buscá si aparece algo como:

'totalCharges'

'Total_Charges'

'total_charges'

'chargesTotal'

🛠 2. Corregir el nombre en tu código
Cuando encuentres el nombre correcto, ajustá tu código. Por ejemplo, si la columna se llama 'totalCharges':

python
Copiar
Editar
df['totalCharges'] = pd.to_numeric(df['totalCharges'], errors='coerce')
📌 3. ¿La columna estaba dentro de un campo tipo diccionario?
Si tu DataFrame original tiene columnas anidadas (como customer, account, etc.), puede que 'TotalCharges' esté dentro de alguna de ellas. En ese caso, necesitás "expandir" la columna tipo dict.

Ejemplo:

python
Copiar
Editar
# Expandir la columna 'account' (si contiene 'TotalCharges')
cuenta_expandida = pd.json_normalize(df['account'])

# Verificá las columnas
print(cuenta_expandida.columns)

# Unila al DataFrame original
df = pd.concat([df.drop(columns=['account']), cuenta_expandida], axis=1)
✅ ¿Querés que revise y expanda automáticamente todas las columnas tipo dict?
Puedo darte un script que detecta, expande y une todo sin que tengas que hacerlo manualmente. ¿Te lo preparo?








Dijiste:
si


ChatGPT dijo:
Perfecto, Adrian. Acá tenés un script automático en Google Colab que:

Detecta las columnas que contienen diccionarios (objetos tipo dict).

Expande cada una en nuevas columnas.

Une todo en un único DataFrame limpio (df_limpio).

Elimina las columnas originales anidadas.

✅ Código para expandir columnas tipo dict y unirlas al DataFrame
python
Copiar
Editar
import pandas as pd

# Suponiendo que ya tenés tu DataFrame cargado como `df`
df_limpio = df.copy()

# Paso 1: Detectar columnas tipo dict
columnas_dict = [col for col in df.columns if isinstance(df[col].dropna().iloc[0], dict)]

# Paso 2: Expandir cada columna dict
for col in columnas_dict:
    # Normalizar la columna (expandir en columnas separadas)
    columnas_expandidas = pd.json_normalize(df_limpio[col])

    # Renombrar columnas para evitar conflictos
    columnas_expandidas.columns = [f"{col}.{subcol}" for subcol in columnas_expandidas.columns]

    # Agregar las columnas expandidas al DataFrame original
    df_limpio = pd.concat([df_limpio.drop(columns=[col]), columnas_expandidas], axis=1)

# Confirmar columnas nuevas
print("Columnas después de la expansión:")
print(df_limpio.columns.tolist())

# Revisar una muestra
df_limpio.head()

In [ ]:
# Generar resumen de columnas que no son tipo dict o lista
columnas_validas = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, (dict, list))).sum() == 0]

resumen = pd.DataFrame({
    'Tipo': df[columnas_validas].dtypes,
    'Nulos (%)': df[columnas_validas].isnull().mean() * 100,
    'Únicos': df[columnas_validas].nunique()
})

# Mostrar resumen
resumen.sort_values('Nulos (%)', ascending=False)

,Tipo,Nulos (%),Únicos
customerID,object,0.0,7267
Churn,object,0.0,3


In [ ]:
# Asegurarse de que la columna de facturación mensual esté en formato numérico
df_limpio['MonthlyCharges'] = pd.to_numeric(df_limpio['MonthlyCharges'], errors='coerce')

# Crear la columna con valor diario aproximado
df_limpio['Cuentas_Diarias'] = df_limpio['MonthlyCharges'] / 30

# Verificamos que se haya creado correctamente
df_limpio[['MonthlyCharges', 'Cuentas_Diarias']].head()

NameError: name 'df_limpio' is not defined

In [ ]:
import pandas as pd

df_limpio = df.copy()

# Detectar columnas con diccionarios
columnas_dict = [col for col in df.columns if isinstance(df[col].dropna().iloc[0], dict)]

# Expandir y unir columnas tipo dict
for col in columnas_dict:
    columnas_expandidas = pd.json_normalize(df_limpio[col])
    columnas_expandidas.columns = [f"{col}.{subcol}" for subcol in columnas_expandidas.columns]
    df_limpio = pd.concat([df_limpio.drop(columns=[col]), columnas_expandidas], axis=1)

In [ ]:
df_limpio['MonthlyCharges'] = pd.to_numeric(df_limpio['MonthlyCharges'], errors='coerce')
df_limpio['Cuentas_Diarias'] = df_limpio['MonthlyCharges'] / 30

NameError: name 'df_limpio' is not defined

In [ ]:
df_limpio.columns.tolist()

In [ ]:
# Asegurarse de que la columna exista
df_limpio['account.MonthlyCharges'] = pd.to_numeric(df_limpio['account.MonthlyCharges'], errors='coerce')

# Calcular cuentas diarias
df_limpio['Cuentas_Diarias'] = df_limpio['account.MonthlyCharges'] / 30

# Verificar resultado
df_limpio[['account.MonthlyCharges', 'Cuentas_Diarias']].head()

NameError: name 'df_limpio' is not defined

In [ ]:
df_limpio.columns.tolist()

NameError: name 'df_limpio' is not defined

In [ ]:
# Asegurarse de que la columna de cargos mensuales sea numérica
df_limpio['account.Charges.Monthly'] = pd.to_numeric(df_limpio['account.Charges.Monthly'], errors='coerce')

# Crear la columna 'Cuentas_Diarias'
df_limpio['Cuentas_Diarias'] = df_limpio['account.Charges.Monthly'] / 30

# Verificar que todo esté correcto
df_limpio[['account.Charges.Monthly', 'Cuentas_Diarias']].head()


NameError: name 'df_limpio' is not defined

✅ 1. Convertir "Sí"/"No" a valores binarios (1/0)

In [ ]:
# Convertir múltiples columnas de 'Sí'/'No' a 1/0
cols_binarias = ['customer.Partner', 'customer.Dependents', 'phone.PhoneService',
                 'account.PaperlessBilling']  # ajustá según tus columnas

df_limpio[cols_binarias] = df_limpio[cols_binarias].replace({'Sí': 1, 'No': 0})

NameError: name 'df_limpio' is not defined

✅ 2. Renombrar columnas para mayor claridad

In [ ]:
df_limpio.rename(columns={
    'customer.SeniorCitizen': 'EsAdultoMayor',
    'account.Charges.Monthly': 'CargoMensual',
    'account.Charges.Total': 'CargoTotal',
    'account.PaymentMethod': 'MetodoPago'
}, inplace=True)

✅ 3. Normalizar texto (pasar a minúsculas, eliminar espacios)

In [ ]:
# Por ejemplo, limpiar la columna MetodoPago
df_limpio['MetodoPago'] = df_limpio['MetodoPago'].str.strip().str.lower()

✅ 4. Mapear categorías a valores numéricos

In [ ]:
# Ejemplo: traducir tipo de contrato a números
df_limpio['account.Contract'] = df_limpio['account.Contract'].map({
    'Mes a mes': 0,
    'Un año': 1,
    'Dos años': 2
})

#📊 Carga y análisis



1. Estadísticas descriptivas básicas (numéricas)

In [ ]:
# Resumen estadístico para columnas numéricas
df_limpio.describe().T

2. Mediana para columnas numéricas

In [ ]:
df_limpio.median(numeric_only=True)

3. Estadísticas para variables categóricas (frecuencias)

In [ ]:
# Por ejemplo, para la columna 'Churn' (evasión)
df_limpio['Churn'].value_counts(normalize=True) * 100  # Porcentaje de cada categoría

4. Un resumen completo personalizado

In [ ]:
import pandas as pd

def resumen_estadistico(df):
    resumen = pd.DataFrame()
    resumen['media'] = df.mean(numeric_only=True)
    resumen['mediana'] = df.median(numeric_only=True)
    resumen['std'] = df.std(numeric_only=True)
    resumen['min'] = df.min(numeric_only=True)
    resumen['max'] = df.max(numeric_only=True)
    resumen['nulos (%)'] = df.isnull().mean() * 100
    resumen['valores_unicos'] = df.nunique()
    return resumen

resumen = resumen_estadistico(df_limpio)
print(resumen)

Opción 1: Gráfico de barras con porcentajes (matplotlib)

In [ ]:
import matplotlib.pyplot as plt

# Contar valores y calcular porcentajes
counts = df_limpio['Churn'].value_counts()
percent = counts / counts.sum() * 100

# Crear gráfico de barras
plt.figure(figsize=(6,4))
bars = plt.bar(counts.index, counts.values, color=['skyblue', 'salmon'])
plt.title('Distribución de Churn (Evasión)')
plt.xlabel('Estado')
plt.ylabel('Cantidad de clientes')

# Añadir etiquetas con porcentaje sobre cada barra
for bar, p in zip(bars, percent):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{p:.1f}%', ha='center', va='bottom')

plt.show()

Opción 2: Gráfico de pastel (pie chart)

In [ ]:
plt.figure(figsize=(6,6))
plt.pie(percent, labels=percent.index, autopct='%1.1f%%', colors=['skyblue', 'salmon'], startangle=90)
plt.title('Proporción de clientes según Churn')
plt.show()

#📄Informe final

